
# Classification Evaluation Metrics


In [1]:

fileloc  = '/mnt/batch/tasks/shared/LS_root/mounts/clusters/compute-one/code/Users/karan.bhatti'

import sys
print(sys.path)
print()
#sys.path.append(f'{fileloc}')
print()
#print(sys.path)


['/mnt/batch/tasks/shared/LS_root/mounts/clusters/compute-one/code/Users/karan.bhatti', '/anaconda/envs/azureml_py38/lib/python38.zip', '/anaconda/envs/azureml_py38/lib/python3.8', '/anaconda/envs/azureml_py38/lib/python3.8/lib-dynload', '', '/anaconda/envs/azureml_py38/lib/python3.8/site-packages', '/anaconda/envs/azureml_py38/lib/python3.8/site-packages/azureml/_project/vendor', '/anaconda/envs/azureml_py38/lib/python3.8/site-packages/azureml/automl/core/_vendor']




In [2]:

class Data(object):
    pass

df = Data
summ = Data


In [3]:
import helpdesk as hd

In [4]:

import pandas as pd, numpy as np
from datetime import datetime

import random


In [5]:

DEBUG = True
seed = 1919


In [6]:

pd.set_option('display.max_rows', 50);
pd.set_option('display.max_columns', 50);
pd.set_option('display.width', 200);


In [34]:

preds_dataset_name = 'predictions_dataset_2022-10-18.csv'

df_eval_raw = pd.read_csv(f'{fileloc}/{preds_dataset_name}')

print(df_eval_raw.shape)
print()
print(df_eval_raw.columns.values.tolist())


(243382, 16)

['@id_DOG', 'DogName', 'y_runtime', 'TrainerName', '@id_RACE', 'RaceNum', 'RaceName', 'Track', 'x_dist', 'x_races_tot', 'position', 'ft_sp', 'x_no_prev_races', 'x_avg_speed_prev', 'ttflag', 'preds']


In [38]:

df_num_dogs = df_eval_raw.groupby("@id_RACE").agg( { '@id_DOG':'nunique' } ).reset_index().rename(columns = {'@id_DOG':'x_num_dogs'})

df_evaluation = pd.merge(df_eval_raw, df_num_dogs, on = ["@id_RACE"], how = 'left')

print(df_evaluation.x_num_dogs.value_counts())
    

8    130968
7     58681
6     38166
5     12575
4      2408
3       378
2       124
1        82
Name: x_num_dogs, dtype: int64


In [39]:

df_evaluation.isna().sum()


@id_DOG                0
DogName                0
y_runtime              0
TrainerName            0
@id_RACE               0
RaceNum                0
RaceName               0
Track                  0
x_dist                 0
x_races_tot            0
position               0
ft_sp               1200
x_no_prev_races        0
x_avg_speed_prev       0
ttflag                 0
preds                  0
x_num_dogs             0
dtype: int64

In [41]:

groupings = ['@id_RACE']

# Rank for Place Preds
df_evaluation["position_preds"] = df_evaluation.groupby( groupings )["preds"].rank("dense", ascending=False)

# Place Preds Columns
df_evaluation['Preds_top3'] = df_evaluation.apply(lambda x : 1 if x.position_preds >= 3 else 0, axis = 1)
df_evaluation['Result_top3'] = df_evaluation.apply(lambda x : 1 if x.position <=3 else 0, axis = 1)



In [42]:

df_evaluation.sort_values(by = ['@id_RACE','position'])


,@id_DOG,DogName,y_runtime,TrainerName,@id_RACE,RaceNum,RaceName,Track,x_dist,x_races_tot,position,ft_sp,x_no_prev_races,x_avg_speed_prev,ttflag,preds,x_num_dogs,position_preds,Preds_top3,Result_top3
33455,514250448,ZIPPY TESLA,29.79,A Dailly,744352338,8,TAB SILVER CHIEF FINAL,Meadows,525,0,1,5.8,0,17306.720264,Train,30.667002,8,3.0,1,1
207642,524387692,COMPLIANCE,29.80,K Pitt,744352338,8,TAB SILVER CHIEF FINAL,Meadows,525,0,2,8.1,0,17442.331767,Test,30.635052,8,7.0,1,1
127850,541738818,TITAN BLAZER,29.96,D Gibbons,744352338,8,TAB SILVER CHIEF FINAL,Meadows,525,0,3,6.2,0,17428.584995,Train,30.638287,8,6.0,1,1
143989,481007104,WEBLEC BLAZER,30.13,A Langton,744352338,8,TAB SILVER CHIEF FINAL,Meadows,525,0,4,9.2,0,17326.623272,Train,30.662310,8,4.0,1,0
230002,580792932,TIGGERLONG NIBEL,30.15,S Grenfell,744352338,8,TAB SILVER CHIEF FINAL,Meadows,525,0,5,21.5,0,16555.124274,Test,30.834932,8,1.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128845,543080498,DERRY GIRL,20.04,A Hyde,829251704,9,TRIALS EVERY TUESDAY (1-2 WINS),Healesville,350,15,4,13.1,0,17074.574041,Train,20.581380,8,1.0,0,0
82531,620538683,SPEEDY MAGPIE,20.12,B Galea,829251704,9,TRIALS EVERY TUESDAY (1-2 WINS),Healesville,350,7,5,4.9,0,17411.223898,Train,20.558820,8,3.0,1,0
145665,652278956,KING POP,20.12,K Tabb,829251704,9,TRIALS EVERY TUESDAY (1-2 WINS),Healesville,350,12,6,2.0,0,17332.728484,Train,20.556181,8,4.0,1,0
13401,502915323,BAFFLING,20.19,R Dunne,829251704,9,TRIALS EVERY TUESDAY (1-2 WINS),Healesville,350,42,7,24.8,0,17080.187196,Train,20.512634,8,7.0,1,0


In [43]:

pd.crosstab(df_evaluation.position, df_evaluation.position_preds)


position_preds,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0
position,,,,,,,,
1,3268,3813,4291,4904,5431,5366,4361,3128
2,3642,4351,4786,5129,5276,4963,3771,2529
3,4051,4621,5044,5168,4997,4655,3643,2183
4,4657,5004,5146,5089,4866,4242,3218,2002
5,5232,5157,4953,4785,4605,3987,3043,1856
6,5447,4837,4561,4171,3887,3714,2747,1723
7,4708,4031,3442,3087,2886,2592,2464,1519
8,3506,2624,2159,1909,1706,1595,1493,1361


In [44]:

df_evaluation.groupby( [ 'ttflag', 'Result_top3', 'Preds_top3']).agg({'@id_DOG':'count'}).reset_index()


,ttflag,Result_top3,Preds_top3,@id_DOG
0,Test,0,0,13629
1,Test,0,1,28385
2,Test,1,0,7027
3,Test,1,1,23974
4,Train,0,0,31574
5,Train,0,1,66423
6,Train,1,0,16719
7,Train,1,1,55651


In [22]:

pd.crosstab(df_evaluation.Result_top3, df_evaluation.Preds_top3, normalize='all')


Preds_top3,0,1
Result_top3,,
0,0.185729,0.389544
1,0.097567,0.327161


In [45]:

df_evaluation


,@id_DOG,DogName,y_runtime,TrainerName,@id_RACE,RaceNum,RaceName,Track,x_dist,x_races_tot,position,ft_sp,x_no_prev_races,x_avg_speed_prev,ttflag,preds,x_num_dogs,position_preds,Preds_top3,Result_top3
0,460023716,LOCKNANE SHELL,26.30,A Bullock,792243445,7,JARED DAVIES CARPENTRY & MAINTENANCE,Hobart,461,29,2,14.00,0,17204.879071,Train,26.879677,7,6.0,1,1
1,561782099,OLDRIK BALE,26.23,M Delbridge,753395293,11,PUP TO TRACK,Geelong,460,2,4,1.80,0,17526.926578,Train,26.849634,5,4.0,1,0
2,580794355,UP AN ATOM,16.94,G Hall,771199464,3,DGR.DOG,Cannington,275,10,7,11.90,0,16516.893108,Train,16.571054,8,6.0,1,0
3,546258339,BANDERILLERO,17.22,W Boreland,817651067,1,HIGHWAY 31 MAIDEN CHALLENGE HT1,Healesville,300,0,4,7.70,0,17712.703076,Train,17.860605,8,2.0,0,0
4,456588895,JOJO MCGRAW,25.95,A Provest,785346597,7,LADBROKES BET TICKER,Bathurst,450,7,1,8.50,0,16965.801040,Train,26.336159,7,2.0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
243377,792289188,PALAWA QUEEN,29.97,E Hughes,822949711,3,LADBROKES MATES MODE MIXED 0-2 WINS,Dubbo,516,1,3,2.15,0,17131.474104,Test,30.157360,4,2.0,0,1
243378,1417590059,WATTA STORM,17.72,J Hewett,767946449,12,RADIO 2LM & TRIPLE Z MASTERS,Casino,300,1,2,15.00,0,16646.415553,Test,17.817347,7,4.0,1,1
243379,1549980031,DAINTREE LILLI,19.20,E Anderson,745589351,9,LADBROKES HANDICAP MASTERS,Bathurst,307,0,8,101.00,0,17152.658662,Test,18.177345,8,6.0,1,0
243380,1858820014,HOLD YOUR FIRE,19.33,A Roche,750418353,6,RANVET PATHWAYS,Taree,314,1,6,61.00,0,16127.375449,Test,18.677350,6,1.0,0,0


In [ ]:
# Strategies


In [51]:

# Bet on all Preds = s1
df_evaluation['fs_1'] = df_evaluation.apply( lambda x : True if x.position_preds == 1 else False, axis = 1 )
df_evaluation['p_s_1'] = df_evaluation.apply( lambda x : x.ft_sp - 1 if ( x.fs_1 == 1 ) & ( x.position == 1 ) else -1 , axis = 1 )


# Bet on all Place = s2
#df_evaluation['fs_2'] = df_evaluation.apply(lambda x : 1 if x.position_preds <= 3 else 0, axis = 1)
df_evaluation["fs_2"] = df_evaluation.apply(lambda x : np.nan if pd.isna(x.position_preds) \
                                                                else True if (x.x_num_dogs >= 8) & (x.position_preds <= 3) \
                                                                    else True if (x.x_num_dogs >= 5) & (x.position_preds <= 2) \
                                                                        else False if (x.x_num_dogs < 5) \
                                                                            else False, axis = 1)

df_evaluation['p_s_2'] = df_evaluation.apply( lambda x : x.ft_sp - 1 if ( x.fs_2 == 1 ) & ( x.position <= 3 ) else -1 , axis = 1 )

print()
print(datetime.now())



2022-10-18 06:17:55.574039


In [52]:

df_evaluation


,@id_DOG,DogName,y_runtime,TrainerName,@id_RACE,RaceNum,RaceName,Track,x_dist,x_races_tot,position,ft_sp,x_no_prev_races,x_avg_speed_prev,ttflag,preds,x_num_dogs,position_preds,Preds_top3,Result_top3,fs_1,p_s_1,fs_2,p_s_2
0,460023716,LOCKNANE SHELL,26.30,A Bullock,792243445,7,JARED DAVIES CARPENTRY & MAINTENANCE,Hobart,461,29,2,14.00,0,17204.879071,Train,26.879677,7,6.0,1,1,False,-1.0,False,-1.0
1,561782099,OLDRIK BALE,26.23,M Delbridge,753395293,11,PUP TO TRACK,Geelong,460,2,4,1.80,0,17526.926578,Train,26.849634,5,4.0,1,0,False,-1.0,False,-1.0
2,580794355,UP AN ATOM,16.94,G Hall,771199464,3,DGR.DOG,Cannington,275,10,7,11.90,0,16516.893108,Train,16.571054,8,6.0,1,0,False,-1.0,False,-1.0
3,546258339,BANDERILLERO,17.22,W Boreland,817651067,1,HIGHWAY 31 MAIDEN CHALLENGE HT1,Healesville,300,0,4,7.70,0,17712.703076,Train,17.860605,8,2.0,0,0,False,-1.0,True,-1.0
4,456588895,JOJO MCGRAW,25.95,A Provest,785346597,7,LADBROKES BET TICKER,Bathurst,450,7,1,8.50,0,16965.801040,Train,26.336159,7,2.0,0,1,False,-1.0,True,7.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
243377,792289188,PALAWA QUEEN,29.97,E Hughes,822949711,3,LADBROKES MATES MODE MIXED 0-2 WINS,Dubbo,516,1,3,2.15,0,17131.474104,Test,30.157360,4,2.0,0,1,False,-1.0,False,-1.0
243378,1417590059,WATTA STORM,17.72,J Hewett,767946449,12,RADIO 2LM & TRIPLE Z MASTERS,Casino,300,1,2,15.00,0,16646.415553,Test,17.817347,7,4.0,1,1,False,-1.0,False,-1.0
243379,1549980031,DAINTREE LILLI,19.20,E Anderson,745589351,9,LADBROKES HANDICAP MASTERS,Bathurst,307,0,8,101.00,0,17152.658662,Test,18.177345,8,6.0,1,0,False,-1.0,False,-1.0
243380,1858820014,HOLD YOUR FIRE,19.33,A Roche,750418353,6,RANVET PATHWAYS,Taree,314,1,6,61.00,0,16127.375449,Test,18.677350,6,1.0,0,0,True,-1.0,True,-1.0


In [53]:

# Classic overestimation of Placement problems -- placing top 3 for 8, and also 3 for 7,6,5,4,3,2,1 racers (fix)
# Classic using WIN price for PLACE issue

df_evaluation.groupby(['ttflag','Track']).agg({ 'fs_1':'sum','p_s_1':'sum','fs_2':'sum','p_s_2':'sum','@id_RACE':'nunique'})



,fs_1,p_s_1,fs_2,p_s_2,@id_RACE
ttflag,,,,,
Test,10274,-65690.69,25089,26004.95,31466
Train,24237,-154429.18,58582,67570.49,34459


In [ ]:

print("THE END")
